In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.gaussian_process.kernels import ConstantKernel, Matern
from sklearn.base import clone
from skopt import gp_minimize
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import ConstantKernel, Matern

In [7]:
def coupling_matrix(N, weights, h=0):
    J = np.zeros((N, N))
    k = 0
    for i in range(N-1):
        #J[i, i] = h/2
        #J[i,i+1] = -1
        for j in range(N-1, i, -1):
            J[i,j] = weights[k]
            k = k + 1
    J = J + np.transpose(J)
    
    # ensuring that J is symmetric
    
    return J

In [118]:
def definitions(N, Tmax):  
    # N is the total number of spins
    # Tmax is the total number of evolutions

    x = np.zeros((N, Tmax))   #the value of the voltages
    f = np.zeros((N, Tmax))   #the function f
    g = np.random.normal(0, 0.01, size=(N, Tmax)) #gaussian noise

    # all the above three arrays have been defined such that 
    # the second dimension corresponds to different number of evolutions
    # the first dimension includes different spins 

    J = coupling_matrix(N, weights)

    return x, f, g, J

def voltage(alpha, beta, N, Tmax, x, f, g, J, plot=0):  
    for k in range(Tmax-1):
        for n in range(N):
            sum1 = 0
            if beta != 0:
                for j in range(N):
                    sum1 += J[j, n]*x[j, k]
            f[n, k] = alpha*x[n, k] + beta*sum1                              #equation 3.2 in the paper with beta=0
            x[n, k+1] = (np.cos(f[n, k] + g[n, k] - np.pi/4))**2 - 0.5       #equation 3.1 in the paper
        if plot==1:
            spins = np.sign(x[:, k+1])
            plt.imshow(spins.reshape(10,10))
            plt.show()
    return x

In [9]:
def fun(N, alpha, beta):
    Tmax = 50
    flag = 0
    y = np.zeros(1)
    
    for k in range(5):
        x, f, g, J = definitions(N, Tmax)
        x = voltage(alpha, beta, N, Tmax, x, f, g, J)
        if x[0,-1] > 0:
            flag = 1
        
        for i in range(1,N,1):
            if J[i-1, i] < 0:
                if x[i, -1] > 0:
                    if flag == 1:
                        y[0] += 1
                    flag = 1
                else:
                    if flag == -1:
                        y[0] += 1
                    flag = -1
            if J[i-1, i] > 0:
                if x[i, -1] > 0:
                    if flag == -1:
                        y[0] += 1
                    flag = 1
                else:
                    if flag == 1:
                        y[0] += 1
                    flag = -1
    y /= 0.05*N
    y = 100 - y
    return y

def energy(Tmax, N, alpha, beta, array=0):
    x, f, g, J = definitions(N, Tmax)
    x = voltage(alpha, beta, N, Tmax, x, f, g, J)
    E = np.zeros(Tmax)
    for k in range(Tmax):
        for i in range(N):
            for j in range(N):
                E[k] += J[i, j]*np.sign(x[i, k])*np.sign(x[j, k])
    if array == 0:
        return E[-1]
    if array == 1:
        return E

In [10]:
def spins(alpha, beta, N, Tmax):
    x, f, g, J = definitions(N, Tmax)
    x = voltage(alpha, beta, N, Tmax, x, f, g, J)
    spins = np.sign(x[:, -1])
    return spins

In [11]:
def plot_approximation(gpr, X, X_sample, Y_sample, X_next=None, show_legend=False):
    mu, std = gpr.predict(X, return_std=True)
    plt.fill_between(X.ravel(), 
                     mu.ravel() + 1.96 * std, 
                     mu.ravel() - 1.96 * std, 
                     alpha=0.1) 
    #plt.plot(X, Y, 'y--', lw=1, label='Objective')
    plt.plot(X, mu, 'b-', lw=1, label='Surrogate function')
    plt.plot(X_sample, Y_sample, 'kx', mew=3, label='samples')
    if X_next:
        plt.axvline(x=X_next, ls='--', c='k', lw=1)
    if show_legend:
        plt.legend()

In [125]:
noise = 0.2
bounds = np.array([[0.2/np.tan(82*np.pi/180), 0.4]])

In [13]:
import warnings
warnings.filterwarnings("ignore")

Optimal values that give accurate anti-ferromagnetic spins only considering nearest neighbour interactions in a periodic 1 dimensional lattice with connected endpoints.

In [ ]:
X = np.arange(0,2.1,0.1).reshape(-1,1) # beta values
Z = np.arange(0,7.1,0.1) # alpha values    
Y = np.zeros(21).reshape(-1,1)
X_init = np.array([[0.4], [1.8]])
Y_init = np.zeros(2).reshape(-1, 1)
n_calls = 5
N = 100

In [ ]:
#optimal_values = [[[], []],[[], []], [[], []], [[], []], [[], []]] 
optimal_values = [[], [], []]
                            #The first row stores the value of beta
                            #The second row stores the value of the accuracy 
                            #The third row stores the spins itself
                            #The third dimension is for different total number of spins

start = time.time()
for z_index in range(len(Z)):
    n52 = ConstantKernel(1.0) * Matern(length_scale=1.5, length_scale_bounds=(1e-150, 10), nu=2.5)
    gpr = GaussianProcessRegressor(kernel=n52, alpha=noise**2)
    Y_init[0] = fun(N, Z[z_index], X_init[0])
    Y_init[1] = fun(N, Z[z_index], X_init[1])
    r = gp_minimize(lambda x: -fun(N, Z[z_index], np.array(x))[0], 
                    bounds.tolist(),
                    base_estimator=gpr,
                    acq_func='EI',      # expected improvement
                    xi=0.5,            # exploitation-exploration trade-off
                    n_calls=n_calls,         # number of iterations
                    n_random_starts=0,  # initial samples are provided
                    x0=X_init.tolist(), # initial samples
                    y0=-Y_init.ravel())

    # Fit GP model to samples for plotting results
    gpr.fit(r.x_iters, -r.func_vals)

    # Plot the fitted model and the noisy samples
    #plt.title("alpha = {}".format(Z[z_index]))
    #plot_approximation(gpr, X, r.x_iters, -r.func_vals, show_legend=True)
    #plt.show()
    
    #Storing the optimal values of beta for a given alpha
    optimal_values[0].append(r.x_iters[np.argmax(-r.func_vals)][0])
    optimal_values[1].append(np.max(-r.func_vals))
    
    Tmax = 50
    x, f, g, J = definitions(N, Tmax)
    x = voltage(Z[z_index], optimal_values[0][-1], N, Tmax, x, f, g, J)
    spins = np.sign(x[:, -1])
    optimal_values[2].append(spins)
    #acc.append(-r.func_vals)
    #beta.append(r.x_iters)
    
end = time.time()
print("Time = {}s".format(end-start))

In [ ]:
optimal_values[1][4]